# Classificador CASCADE

* Precisa de grupo de imagens positivas (no caso de detecção de faces, ter apenas faces), e, um grupo negativo (não faces)

* Passando pelo algoritmo [AdaBoost](https://www.machinelearningplus.com/machine-learning/introduction-to-adaboost/) (recebe a base de dados e detecta o padrão)

    * It works in the following steps:
        1. Initially, Adaboost selects a training subset randomly;
        2. It iteratively trains the AdaBoost machine learning model by selecting the training set based on the accurate prediction of the last training;
        3. It assigns the higher weight to wrong classified observations so that in the next iteration these observations will get the high probability for classification;
        4. Also, It assigns the weight to the trained classifier in each iteration according to the accuracy of the classifier. The more accurate classifier will get high weight
        5. This process iterates until the complete training data fits without any error or until reached to the specified maximum number of estimators


* Classificador [CASCADE](https://en.wikipedia.org/wiki/Cascading_classifiers) consiste em uma série de classificadores que identificam características individuais de um objeto, e, retornam em série os resultados de cada classificar. Caso todos os resultados forem positivos, o classificador CASCADE classifica o objeto como positivo.

In [2]:
# python -m pip install numba cudatoolkit
from numba import jit, cuda 
import numpy as np 
# to measure exec time 
from timeit import default_timer as timer    
  
# normal function to run on cpu 
def func(a):                                 
    for i in range(10000000): 
        a[i]+= 1      
  
# function optimized to run on gpu  
@jit(target_backend='cuda')                          
def func2(a): 
    for i in range(10000000): 
        a[i]+= 1
if __name__=="__main__": 
    n = 10000000                            
    a = np.ones(n, dtype = np.float64) 
      
    start = timer() 
    func(a) 
    print("without GPU:", timer()-start)     
      
    start = timer() 
    func2(a) 
    print("with GPU:", timer()-start) 

ModuleNotFoundError: No module named 'numba'

# Importando bibliotecas

In [1]:
import os
import cv2
import numpy as np

from cv2.typing import MatLike
base_dir = os.path.join(os.getcwd(), 'datasets', 'playground', 'curso')

In [2]:
def display_image(window_name: str, image: MatLike):
    cv2.imshow(window_name, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def convert_image_color(image: MatLike, color: int = cv2.COLOR_BGR2GRAY):
    return cv2.cvtColor(image, color)

## Testando o opencv

In [27]:
# Imagens podem ser encontradas em: https://drive.google.com/drive/folders/1AyJto3_9yWuR5JujFHE15-TZOt6uygzF?usp=sharing
images_dir_path = os.path.join(
    base_dir, 'Images'
)

image = cv2.imread(os.path.join(images_dir_path, 'people1.jpg'))
image.shape

(1280, 1920, 3)

In [28]:
display_image('image', image)

In [7]:
# Reduz o tamanho para facilitar o processamento
image = cv2.resize(image, (800, 600)) 
display_image('image', image)

In [8]:
gray_image = convert_image_color(image)
display_image('gray_image', gray_image)

# Detecção de Faces - HaarCascade

In [17]:
# Carregando o classificador Cascade pre-definido pelo OpenCV
cascade_dir = os.path.join(
    base_dir, 'Cascades'
)

face_detector = cv2.CascadeClassifier(
    os.path.join(cascade_dir, 'haarcascade_frontalface_default.xml')
)


In [22]:
deteccoes = face_detector.detectMultiScale(
    gray_image,
)

# Indica a posicao do objeto na imagem em que pode ser encontrada. Também indica largura e altura
deteccoes

array([[390, 323,  56,  56],
       [387, 233,  73,  73],
       [ 92, 239,  66,  66],
       [115, 124,  53,  53],
       [475, 123,  59,  59],
       [677,  72,  68,  68]])

In [25]:
for deteccao in deteccoes:
    x = deteccao[0]
    y = deteccao[1]
    w = deteccao[2]
    h = deteccao[3]

    initial_pos = (x, y)
    rect_color_bgr = (0, 255, 255)
    rect_thickness = 3

    cv2.rectangle(image, initial_pos, (x + w, y + h), rect_color_bgr, rect_thickness)
    
display_image('image', image)

## Parâmetros HaarCascade

In [54]:
images_dir_path = os.path.join(base_dir, 'Images')
image = cv2.imread(os.path.join(images_dir_path, 'people2.jpg'))
image = cv2.resize(image, (800, 600)) 
gray_image = convert_image_color(image)

cascade_dir = os.path.join(base_dir, 'Cascades')
face_detector = cv2.CascadeClassifier(os.path.join(cascade_dir, 'haarcascade_frontalface_default.xml'))
guesses = face_detector.detectMultiScale(
    image=gray_image,
    # Indica que aumenta a escala da imagem. Util quando possui faces/objetos pequenos na imagem
    # Diminuir o valor de escala quando a imagem for muito pequena. Não pode ser menor que 1.
    scaleFactor=1.1,
    # Número mínimo de vizinhos para considerar uma face, uma face. 
    # O algoritmo irá gerar 'minNeighbors' previsoes de um objeto e irá escolher a que melhor se adapta
    # Maior o valor, maior será a certeza que uma face não é um falso positivo. 
    ## Em contra partida, pode acontecer que faces não sejam encontradas.
    minNeighbors = 2,
    # Tamanho mínimo que o objeto pode ser detectado.
    minSize = (32,32),
    # Tamanho mínimo que o objeto pode ser detectado.
    maxSize = (100, 100)
)

for (x, y, w, h) in guesses:
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

display_image('image', image)


## Detecção de Olhos

In [3]:
images_dir_path = os.path.join(base_dir, 'Images')
image = cv2.imread(os.path.join(images_dir_path, 'people1.jpg'))
# image = cv2.resize(image, (800, 600)) 
gray_image = convert_image_color(image)

cascade_dir = os.path.join(base_dir, 'Cascades')
face_detector = cv2.CascadeClassifier(os.path.join(cascade_dir, 'haarcascade_frontalface_default.xml'))
eye_detector = cv2.CascadeClassifier(os.path.join(cascade_dir, 'haarcascade_eye.xml'))
guesses = face_detector.detectMultiScale(
    image=gray_image,
    # Indica que aumenta a escala da imagem. Util quando possui faces/objetos pequenos na imagem
    # Diminuir o valor de escala quando a imagem for muito pequena. Não pode ser menor que 1.
    scaleFactor=1.3,
    # Número mínimo de vizinhos para considerar uma face, uma face. 
    # O algoritmo irá gerar 'minNeighbors' previsoes de um objeto e irá escolher a que melhor se adapta
    # Maior o valor, maior será a certeza que uma face não é um falso positivo. 
    ## Em contra partida, pode acontecer que faces não sejam encontradas.
    minNeighbors = 2,
    # Tamanho mínimo que o objeto pode ser detectado.
    minSize = (32,32),
    # Tamanho mínimo que o objeto pode ser detectado.
    maxSize = (200, 200)
)

for (x, y, w, h) in guesses:
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

eye_guesses = eye_detector.detectMultiScale(
    image=gray_image,
    scaleFactor=1.09,
    minNeighbors = 10,
    maxSize = (70, 70)
)

for (x, y, w, h) in eye_guesses:
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)

display_image('image', image)